<a href="https://colab.research.google.com/github/arkadijshmelev/Frog_family_classification/blob/master/Frog_family_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Библиотеки

In [33]:
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import linear_model

## Данные

In [34]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [35]:
# Загрузим данные с google drive.
data = drive.CreateFile({'id': '1VXQWWFJt5lJo5ALcSPlvQSUZJnM03qQk'})
data.GetContentFile('Frogs_MFCCs.csv')

In [36]:
# Загрузим данные.
df = pd.read_csv('Frogs_MFCCs.csv')
# Выведем для просмотра.
display(df.head())
# Выведем информацию.
print(df.info())

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7195 entries, 0 to 7194
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MFCCs_ 1  7195 non-null   float64
 1   MFCCs_ 2  7195 non-null   float64
 2   MFCCs_ 3  7195 non-null   float64
 3   MFCCs_ 4  7195 non-null   float64
 4   MFCCs_ 5  7195 non-null   float64
 5   MFCCs_ 6  7195 non-null   float64
 6   MFCCs_ 7  7195 non-null   float64
 7   MFCCs_ 8  7195 non-null   float64
 8   MFCCs_ 9  7195 non-null   float64
 9   MFCCs_10  7195 non-null   float64
 10  MFCCs_11  7195 non-null   float64
 11  MFCCs_12  7195 non-null   float64
 12  MFCCs_13  7195 non-null   float64
 13  MFCCs_14  7195 non-null   float64
 14  MFCCs_15  7195 non-null   float64
 15  MFCCs_16  7195 non-null   float64
 16  MFCCs_17  7195 non-null   float64
 17  MFCCs_18  7195 non-null   float64
 18  MFCCs_19  7195 non-null   float64
 19  MFCCs_20  7195 non-null   float64
 20  MFCCs_21  7195 non-null   floa

Будем решать задачу бинарной классификации по выявлению лягушек, которые относятся к семейству 'Dendrobatidae' (признак 'Family'). Семейство 'Dendrobatidae' будет классом 1, все остальные семейства — классом 0.

In [37]:
df['Family'] = df['Family'].apply(lambda x: 1 if x == 'Dendrobatidae' else 0)

In [38]:
# Создадим матрицу наблюдения необходимых нам признаков.
X = df.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
# Создадим вектор правильных ответов.
y = df['Family']

In [39]:
# Разобьем данные не тренировочную и тестовую выборки.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)

## Случайный лес

In [40]:
# Создадим модель случайного леса.
rf_model = ensemble.RandomForestClassifier(n_estimators=10, random_state=42)
# Обучим модель.
rf_model.fit(X_train, y_train)
# Делаем предсказание.
y_pred_test = rf_model.predict(X_test)
# Выводим метрику.
print('f1_score : {:.2f}'.format(metrics.f1_score(y_test, y_pred_test)))

f1_score : 0.97


## Стекинг

In [41]:
# Составим список базовых моделей.
estimators = [('rf', ensemble.RandomForestClassifier(n_estimators=10, random_state=31)),
              ('knn', neighbors.KNeighborsClassifier(n_neighbors= 11)),
              ('nb', naive_bayes.GaussianNB() )               
]
# Иницианализируем модель StackingClassifier().
metamodel = ensemble.StackingClassifier(
    estimators=estimators, final_estimator=linear_model.LogisticRegression()
)
# Обучим модель.
metamodel.fit(X_train, y_train)
# Делаем предсказание.
y_pred_test = metamodel.predict(X_test)
# Выводим метрику.
print('f1_score : {:.2f}'.format(metrics.f1_score(y_test, y_pred_test)))

f1_score : 0.99
